## Imports

In [1]:
import numpy as np
import polycg

/home/pol_schiessel/ensk996e/.local/lib/python3.11/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


### Generate example RBP parameters
The groundstate and stiffness matrix for a given sequence can be generated with the method `gen_params`. The parameter set can be specified with the argument `method`, with the options:

'lankas': [Lankas et al.](https://doi.org/10.1016/S0006-3495(03)74710-9)

'olson': [Olson et al.](https://www.pnas.org/doi/full/10.1073/pnas.95.19.11163)

'cgnaplus': [Sharma et al.](https://doi.org/10.1016/j.jmb.2023.167978)


`gen_params` return a dictionary that contains the ground state `gs` (expressed in radians (rotations) and nm (translations)), the stiffness matrix `stiff` and the `sequence`. The resulting rotational parameters are expressed as Euler vectors and translations are expressed in terms of the first respective triad. 

In [17]:
# generate random sequence
nbp = 801
seq = ''.join(['atcg'[np.random.randint(4)] for i in range(nbp)])

method = 'cgnaplus'
# method = 'lankas'
# method = 'olson'

# generate stiffness matrix and groundstate with cgnaplus
params = polycg.gen_params('cgnaplus',seq)
gs = params['gs']
stiff = params['stiff']

print(gs.shape)
print(stiff.shape)

Generating partial stiffness matrix with
block_size:   120
overlap_size: 20
tail_size:    20
Generating stiffness from bps 0 to 120 (800 in total).
Generating stiffness from bps 100 to 220 (800 in total).
Generating stiffness from bps 200 to 320 (800 in total).
Generating stiffness from bps 300 to 420 (800 in total).
Generating stiffness from bps 400 to 520 (800 in total).
Generating stiffness from bps 500 to 620 (800 in total).
Generating stiffness from bps 600 to 800 (800 in total).
convert to sparse
(800, 6)
(4800, 4800)


### Coarse-Grain 

In [20]:
# set composite size (k)
composite_size = 10

# set lower and upper bound for restricted selection range
start_id = 0    # starting at first element
end_id   = None # till end

# allow for coarse-graining in partial ranges to avoid large matrix inversion and reduce computation time
allow_partial=True

# size of partial blocks in number of composites
block_ncomp=20 #(block_ncomp*composite_size bps entries)

# size of block overlap to account for non-local couplings
overlap_ncomp=6

# size of additional entries to improve agreement with full calculation if couplings are non-local
tail_ncomp=6

# coarse-grain parameters
cg_gs, cg_stiff = polycg.coarse_grain(
    gs,
    stiff,
    composite_size,
    start_id=start_id,
    end_id=end_id,
    allow_partial=allow_partial,
    block_ncomp=block_ncomp,
    overlap_ncomp=overlap_ncomp,
    tail_ncomp=tail_ncomp
)

print(cg_gs.shape)
print(cg_stiff.shape)


# coarse-grain without partial
allow_partial = False
cg_gs, cg_stiff_np = polycg.coarse_grain(gs,stiff,composite_size,start_id=start_id,end_id=end_id,allow_partial=allow_partial,block_ncomp=block_ncomp,overlap_ncomp=overlap_ncomp,tail_ncomp=tail_ncomp)


# difference between partial and non-partial
diff = np.sum(np.abs(cg_stiff.toarray()-cg_stiff_np.toarray()))
print(f'difference = {diff}')


Coarse-graining from bps 0 to 200 (800 in total).
Coarse-graining from bps 140 to 340 (800 in total).
Coarse-graining from bps 280 to 480 (800 in total).
Coarse-graining from bps 420 to 620 (800 in total).
Coarse-graining from bps 560 to 800 (800 in total).
(80, 6)
(480, 480)
difference = 0.0007227872403280678


### Coarse-Grain ground state only

In [24]:
composite_size = 10
cg_gs = polycg.cg_groundstate(gs,composite_size=composite_size)
print(cg_gs.shape)

(80, 6)


### Direct Generation
The method `gen_params` can directly generate coarse-grained parameters if if the argument `composite_size` is specified

In [ ]:
# generate random sequence
nbp = 401
seq = ''.join(['atcg'[np.random.randint(4)] for i in range(nbp)])
composite_size = 10

params = polycg.gen_params('cgnaplus',seq,composite_size=composite_size)
cg_gs = params['cg_gs']
cg_stiff = params['cg_stiff']

print(cg_gs.shape)
print(cg_stiff.shape)